# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 20% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [6]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

In [7]:
import os 
os.getcwd()

'/home/jovyan/work/assignments/assignment3'

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [24]:
#def answer_one():
def get_energy():
    # YOUR CODE HERE    
    ###########################
    # Part One
    # Enery Indicators.xls
    # 
    
    energy_df = pd.read_excel('Energy Indicators.xls', skiprows=16)
    del energy_df['Unnamed: 0']
    del energy_df['Unnamed: 1']
    
    # Drop NA data in excel
    energy_df.dropna(inplace=True)
    
    # change the column labels ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy_df.rename(columns={"Unnamed: 2":"Country", "Renewable Electricity Production": "% Renewable", "Energy Supply per capita": "Energy Supply per Capita"}, inplace=True)

    # Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule).
    energy_df.replace('...', np.NaN, inplace=True)

    # For all countries which have missing data (e.g. data with "...") make sure this is reflected as np.NaN values
    energy_df['Energy Supply'] = energy_df['Energy Supply'] * 1000000

    # There are also several countries with numbers and/or parenthesis in their name
    energy_df['Country'] = energy_df['Country'].str.replace(' \(.*\)', '')
    energy_df['Country'] = energy_df['Country'].str.replace('[0-9]*', '')

    # Rename the following list of countries
    x = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
    }

    for item in x:
        energy_df['Country'].replace(item, x[item], inplace=True)
        
    return energy_df


def get_gdp():
    
    # Part TWO
    # world_bank.csv
    gdp_df = pd.read_csv('world_bank.csv', skiprows=4)

    # rename the following list of countries
    y = {
    "Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
    }

    for item in y:
        gdp_df['Country Name'].replace(item, y[item], inplace=True)
    
    gdp_df.rename(columns={'Country Name': 'Country'}, inplace=True)
        
    return gdp_df


def get_rank():
    ###########################
    # Part THREE
    # scimagojr-3.xlsx
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    return ScimEn


def answer_one():
    energy_df = get_energy()
    gdp_df = get_gdp()
    ScimEn = get_rank()
    
    ###########################
    # 
    # Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).

    for i in range(1960, 2006):
        gdp_df.drop([str(i)], axis=1, inplace=True)

    ScimEn = ScimEn[0:15]
    
    # Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names).
    temp1 = pd.merge(energy_df, gdp_df, how="inner", left_on="Country", right_on="Country")
    temp2 = pd.merge(temp1, ScimEn, how="inner")

    # The index of this DataFrame should be the name of the country 
    temp2.set_index('Country', inplace=True)
 
    # the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'].
    list_to_be_drop = ['Country Code', 'Indicator Name', 'Indicator Code']
    for item in list_to_be_drop:
        temp2.drop(item, axis=1, inplace=True)
    
    # Check
#     real = temp2.columns.tolist()
#     shouldbe = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    
#     print(real.sort() == shouldbe.sort())
#     for item in real:
#         if item in shouldbe:
#             print('YES: {} in'.format(item))
#         else:
#             print('NO: {} not in'.format(item))
    
    return temp2    

answer_one()

IndentationError: expected an indented block (<ipython-input-24-4057e4c16cbb>, line 10)

In [13]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


FileNotFoundError: [Errno 2] No such file or directory: 'Energy Indicators.xls'

In [14]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [15]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [16]:
def answer_two():
    # YOUR CODE HERE
    energy_df = get_energy()
    gdp_df = get_gdp()
    rank_df = get_rank()
    
    energy_len = len(energy_df)
#     print(energy_len)
    gdp_len = len(gdp_df)
#     print(gdp_len)
    rank_len = len(rank_df)
#     print(rank_len)
    
    tempAB = pd.merge(energy_df, gdp_df, how="inner")
    merge_AB_len = len(tempAB)
#     print(merge_AB_len)

    tempAC = pd.merge(energy_df, rank_df, how="inner")
    merge_AC_len = len(tempAC)
#     print(merge_AC_len)
    
    tempBC = pd.merge(rank_df, gdp_df, how="inner")
    merge_BC_len = len(tempBC)
#     print(merge_BC_len)
    
    result = energy_len + gdp_len + rank_len - merge_AB_len - merge_AC_len - merge_BC_len
    
#     print(result)
    
    return result

answer_two()

IndentationError: expected an indented block (<ipython-input-16-111d38d0308a>, line 4)

In [17]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


NameError: name 'answer_two' is not defined

### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [20]:
def answer_three():
    # YOUR CODE HERE
    Top15 = answer_one()
    gdp_list = Top15[[str(i) for i in range(2006, 2016)]]
    # The default is to count by column, here to mark axis=1 (by row)
    avgGDP = gdp_list.mean(axis=1).sort_values(ascending=False)
    return avgGDP

answer_three(
    raise NotImplementedError()
    info=answer_one()
    return info[["2006","2007","2008","2009","2010","2011","2012","2013","2014","2015"]].apply(np.mean,axis = 1).sort_values(ascending = False)


IndentationError: expected an indented block (<ipython-input-20-5d555fabd6f1>, line 3)

In [21]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


NotImplementedError: 

### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [25]:
def answer_four():
    # YOUR CODE HERE
    Top15 = answer_one()
    _6th = answer_three().index[5]
    gdp_list = Top15[[str(i) for i in range(2006, 2016)]]
    result = gdp_list.loc[_6th]
    return result[-1] - result[0]

answer_four()

FileNotFoundError: [Errno 2] No such file or directory: 'Energy Indicators.xls'

In [23]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [ ]:
def answer_five():
    # YOUR CODE HERE
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean(axis=0)

answer_five()

In [ ]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [26]:
def answer_six():
    # YOUR CODE HERE
    Top15 = answer_one()
    max = Top15['% Renewable'].max()
    result = Top15[Top15['% Renewable'] == max]
    return (result.index[0], max)

answer_six()

FileNotFoundError: [Errno 2] No such file or directory: 'Energy Indicators.xls'

In [ ]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():
    # YOUR CODE HERE
    Top15 = answer_one()
    total_citations = Top15['Self-citations'].sum()  # Wanye, not a total. .
    Top15['Ratio'] = Top15['Self-citations'] / Top15['Citations']
    max = Top15['Ratio'].max()
    result = Top15[Top15['Ratio'] == max]
    return (result.index[0], max)

answer_seven()

In [ ]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [ ]:
def answer_eight():
    # YOUR CODE HERE
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15.sort_values(by='Population', ascending=False, inplace=True)
    return Top15.iloc[2].name

answer_eight()

In [ ]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    # YOUR CODE HERE
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    result = Top15['Citable docs per Capita'].corr(Top15['Energy Supply per Capita'])
    return result

answer_nine()

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    # YOUR CODE HERE
    Top15 = answer_one()
    median = Top15['% Renewable'].median()
    Top15['HighRenew'] = (Top15['% Renewable'] >= median) * 1
    return Top15.sort_values('Rank')['HighRenew']

answer_ten()

In [ ]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [29]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

def answer_eleven():
    # YOUR CODE HERE    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15_population = Top15['PopEst']
    group = Top15_population.groupby(ContinentDict)
#     size = pd.DataFrame(group.size(), columns=['size'])
#     print(size)
#     sum = pd.DataFrame(group.sum()).rename(columns={'PopEst': 'sum'})
#     print(sum)
#     mean = pd.DataFrame(group.mean()).rename(columns={'PopEst': 'mean'})
#     print(mean)
#     std = pd.DataFrame(group.std()).rename(columns={'PopEst': 'std'})
#     print(std)
#     result = pd.merge(size, sum, left_index=True, right_index=True).merge(mean, left_index=True, right_index=True).merge(std, left_index=True, right_index=True)

    size = group.size().astype('float64')
    sum = group.sum()
    mean = group.mean()
    std = group.std()
    result = pd.DataFrame({
        'size': size,
        'sum': sum,
        'mean': mean,
        'std': std
    })
    return result

answer_eleven()

FileNotFoundError: [Errno 2] No such file or directory: 'Energy Indicators.xls'

In [ ]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [27]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
def answer_twelve():
    # YOUR CODE HERE

    Top15 = answer_one()
    cut = pd.cut(Top15['% Renewable'], bins=5)
    Top15['bin'] = cut
    group = Top15.groupby(by=[ContinentDict, cut])
#     for name, group in result:
#         print(name)
#         print(group)
    result = group.size()
    return result

answer_twelve()

In [ ]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [ ]:
def answer_thirteen():
    # YOUR CODE HERE
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['PopEst'] = Top15['PopEst'].map(lambda x: format(x, ',')) # Source: https://blog.csdn.net/dta0502/article/details/82974541
    result = Top15['PopEst']
    return result

answer_thirteen()

In [ ]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")